# Image Search & LLM Recipe Generation Test

This notebook tests the implementation plan for generating an `imageSearchQuery` using Gemini and retrieving images from Unsplash.

In [1]:
# Install necessary packages
%pip install google-generativeai requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
import requests
import json
from dotenv import load_dotenv
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content
from IPython.display import Image, display  # 모듈 추가


# Load environment variables from .env file
# Make sure you have API_KEY (Gemini) and UNSPLASH_ACCESS_KEY in your .env file
load_dotenv('../.env') 

GEMINI_API_KEY = os.getenv("API_KEY")
UNSPLASH_ACCESS_KEY = os.getenv("UNSPLASH_ACCESS_KEY")

if not GEMINI_API_KEY:
    print("WARNING: API_KEY for Gemini not found in environment.")
if not UNSPLASH_ACCESS_KEY:
    print("WARNING: UNSPLASH_ACCESS_KEY not found in environment.")

genai.configure(api_key=GEMINI_API_KEY)

In [42]:
# Define the Schema with the new 'imageSearchQuery' field

recipe_overview_schema = {
    "type": content.Type.ARRAY,
    "items": {
        "type": content.Type.OBJECT,
        "properties": {
            "recipeName": { "type": content.Type.STRING, "description": "Creative name of the recipe, in the target language." },
            "englishRecipeName": { "type": content.Type.STRING, "description": "The English name of the recipe. Mandatory." },
            "description": { "type": content.Type.STRING, "description": "A short, enticing description in the target language." },
            "imageSearchQuery": { 
                "type": content.Type.STRING, 
                "description": "A concise, optimal search query for finding a relevant image. Focus on main ingredients and dish type. Avoid subjective adjectives (e.g., 'zesty', 'delicious') and abstract terms." 
            },
            "cuisine": { "type": content.Type.STRING, "description": "The type of cuisine." },
            "cookTime": { "type": content.Type.INTEGER, "description": "Estimated cooking time in minutes." },
            "difficulty": { "type": content.Type.STRING, "enum": ["Easy", "Medium", "Hard"] },
            "spiciness": { "type": content.Type.INTEGER, "description": "Spiciness level from 1 to 5." },
            "calories": { "type": content.Type.INTEGER, "description": "Estimated calories per serving." },
            "servings": { "type": content.Type.INTEGER, "description": "Number of servings." },
            "ingredients": { "type": content.Type.ARRAY, "items": { "type": content.Type.STRING }, "description": "List of main ingredient NAMES only (e.g. 'Onion'). Do not include quantities yet. In the target language." },
            "missingIngredients": { "type": content.Type.ARRAY, "items": { "type": content.Type.STRING }, "description": "Names of ingredients the user is missing. In the target language." },
        },
        "required": ["recipeName", "englishRecipeName", "imageSearchQuery", "description", "cuisine", "cookTime", "difficulty", "spiciness", "calories", "servings", "ingredients"]
    }
}

In [43]:
def get_recipe_recommendations(ingredients, priority_ingredients, filters, language='en'):
    model_name = 'gemini-2.5-flash' # Using 2.0 or 1.5 flash as available, logic follows gemini.ts
    # Note: gemini.ts uses 'gemini-2.5-flash' which might be a typo or a specific tuned model. 
    # I'll use 'gemini-1.5-flash' or 'gemini-2.0-flash-exp' if available, or just 'gemini-1.5-flash'.
    # Let's try 'gemini-1.5-flash' for safety as it's standard unless the user has access to newer ones.
    model = genai.GenerativeModel('gemini-2.5-flash')

    target_language = 'Korean' if language == 'ko' else 'English'
    cuisine_filter = 'Any' if filters.get('cuisine') == 'any' else filters.get('cuisine')

    prompt = f"""
      You are an expert chef creating recipes for the "OhMyCook" app.
      
      CONTEXT:
      - User Ingredients: {', '.join(ingredients)}.
      - Priority Ingredients (Must use if possible): {', '.join(priority_ingredients)}.
      
      FILTERS:
      - Cuisine: {cuisine_filter}
      - Servings: {filters.get('servings')}
      - Spiciness: {filters.get('spiciness')}
      - Difficulty: {filters.get('difficulty')}
      - Max Cook Time: {filters.get('maxCookTime')} minutes
      
      TASK:
      Recommend 5 diverse and delicious recipes matching these conditions.
      
      IMPORTANT OUTPUT INSTRUCTIONS:
      1. **Language**: Return all user-facing text (name, description, ingredients list) in **{target_language}**.
      2. **Search Query**: For 'imageSearchQuery', generate a keyword-focused, English string optimized for Image search based on Recipe Name (e.g. "kimchi fried rice").
      3. **Overview Only**: This is the first stage. 
         - For 'ingredients', list ONLY the names (e.g., "Onion", "Pork"). DO NOT include quantities.
         - Do NOT include 'instructions' or 'substitutions' yet.
         - If main ingredients are missing from the user's list, add them to 'missingIngredients'.
    """

    response = model.generate_content(
        prompt,
        generation_config=genai.types.GenerationConfig(
            response_mime_type="application/json",
            response_schema=recipe_overview_schema,
            temperature=0.7
        )
    )
    
    try:
        return json.loads(response.text)
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        print(response.text)
        return []


In [44]:
def search_unsplash_image(query):
    if not UNSPLASH_ACCESS_KEY:
        return "NO_API_KEY"
        
    url = "https://api.unsplash.com/search/photos"
    params = {
        "query": query,
        "per_page": 1,
        "orientation": "landscape"
    }
    headers = {
        "Authorization": f"Client-ID {UNSPLASH_ACCESS_KEY}"
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        results = data.get('results', [])
        if results:
            return results[0]['urls']['regular']
        else:
            return None
    except Exception as e:
        print(f"Unsplash Error for '{query}': {e}")
        return None

In [45]:
def search_google_image(query):
    api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
    cx = os.getenv("GOOGLE_SEARCH_CX")

    if not api_key or not cx:
        return "SKIPPED_NO_KEY"
        
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": api_key,
        "cx": cx,
        "q": query,
        "searchType": "image", # 이미지 검색 모드
        "num": 1,              # 가져올 개수
        "safe": "off"          # 세이프 서치 (필요시 active)
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # items가 존재하면 첫 번째 이미지 링크 반환
        if 'items' in data and len(data['items']) > 0:
            return data['items'][0]['link']
        else:
            return None
    except Exception as e:
        print(f"Google Search Error for '{query}': {e}")
        return None

In [46]:
# TEST EXECUTION

# Mock Data
user_ingredients = ["Beef Sirloin", "Orange", "Rice", "Egg", "Mushroom"]
priority_ingredients = []
filters = {
    "cuisine": "any",
    "servings": 2,
    "spiciness": 1,
    "difficulty": "Easy",
    "maxCookTime": 30
}


print("Generating recipes... (This may take a few seconds)")
recipes = get_recipe_recommendations(user_ingredients, priority_ingredients, filters, language='en')
print(f"Generated {len(recipes)} recipes.\n")
for i, recipe in enumerate(recipes):
    name = recipe.get('recipeName')
    query = recipe.get('imageSearchQuery')
    print(f"{i+1}. {name}")
    print(f"   Image Query: '{query}'")
    
    # Google Search
    image_url1 = search_google_image(name)
    image_url2 = search_google_image(query)
    if image_url1:
        print(f"   Image Found for {name}: {image_url1}")
        display(Image(url=image_url1, width=400)) # <--- 이 부분이 이미지를 띄워줍니다 (width로 크기 조절 가능)
        print(f"   Image Found for {query}: {image_url2}")
        display(Image(url=image_url2, width=400)) # <--- 이 부분이 이미지를 띄워줍니다 (width로 크기 조절 가능)
    else:
        print(f"   No Image Found (or key missing) for '{query}'")
    print("-" * 40)

Generating recipes... (This may take a few seconds)
Generated 5 recipes.

1. Quick Beef and Mushroom Stir-fry
   Image Query: 'Beef mushroom stir fry'
   Image Found for Quick Beef and Mushroom Stir-fry: https://www.jocooks.com/wp-content/uploads/2020/09/beef-mushroom-stir-fry-1-18.jpg


   Image Found for Beef mushroom stir fry: https://www.jocooks.com/wp-content/uploads/2020/09/beef-mushroom-stir-fry-1-18.jpg


----------------------------------------
2. Zesty Orange Beef with Steamed Rice
   Image Query: 'Orange beef rice'
   Image Found for Zesty Orange Beef with Steamed Rice: https://www.akitchenhoorsadventures.com/wp-content/uploads/2014/01/spicy-orange-ginger-beef-stir-fry-12-720x405.jpg


   Image Found for Orange beef rice: https://carlsbadcravings.com/wp-content/uploads/2022/09/orange-beef-10f.jpg


----------------------------------------
3. Savory Beef and Mushroom Fried Rice
   Image Query: 'Beef mushroom fried rice egg'
   Image Found for Savory Beef and Mushroom Fried Rice: https://www.recipetineats.com/uploads/2024/09/Sizzling-Beef-Fried-Rice-54326a.jpg


...

----------------------------------------
4. Beef, Mushroom & Egg Skillet with Rice
   Image Query: 'Beef mushroom egg skillet rice'
   Image Found for Beef, Mushroom & Egg Skillet with Rice: https://beckysbestbites.com/wp-content/uploads/2017/09/Bell-Pepper-Mushroom-and-Ground-Beef-Skillet_-4.jpg


   Image Found for Beef mushroom egg skillet rice: https://beckysbestbites.com/wp-content/uploads/2017/09/Bell-Pepper-Mushroom-and-Ground-Beef-Skillet_-4.jpg


----------------------------------------
5. Refreshing Beef and Orange Rice Bowl
   Image Query: 'Beef orange rice bowl'
   Image Found for Refreshing Beef and Orange Rice Bowl: https://media.hellofresh.com/f_auto,fl_lossy,q_auto,w_1200/hellofresh_s3/image/93bddf52-437d-5376-b4e4-250c7a1067e8-f591a65a.jpg


   Image Found for Beef orange rice bowl: https://www.allrecipes.com/thmb/YivPaZsPOuj5wCM_9SBpJdCtg9A=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/AR-57966-crispy-orange-beef-hero-4x3-a21f2f3cb4ac46a78ebb967de69f4cf9.jpg


----------------------------------------
